In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
juegos = []
with open('/content/drive/My Drive/Chess/filtered_games.pgn', 'r') as f:
    lineas = []
    for line in f:
        if line == "\n":
            continue
        lineas.append(line.strip())
        if len(lineas) == 16:
            juegos.append(lineas)
            lineas = []

In [ ]:
def analizar_fen(fen_str):
    tablero, turno, enroque, enp, halfmove, fullmove = fen_str.split(" ")
    for i in range(1, 9):
        tablero = tablero.replace(str(i), '.' * i)
    return tablero.replace("/", ""), turno, enroque, enp, halfmove, fullmove

In [ ]:
import chess.pgn
from io import StringIO

game_pgn = StringIO("\n".join(juegos[0]))

game = chess.pgn.read_game(game_pgn)

In [ ]:
analizar_fen(game.end().board().fen())

('..K.....q.......P.........k.....................................',
 'w',
 '-',
 '-',
 '4',
 '57')

In [3]:
import chess

tablero = chess.Board()

print(tablero)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [4]:
import numpy as np

def tablero_a_nums(tablero_str):
  # piezas = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
    piezas = ['.', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']
    tablero_array = np.zeros(64)
    for i, elemento in enumerate(tablero_str):
        tablero_array[i] = piezas.index(elemento)
    return tablero_array

def enroque_a_nums(enroque_str):
    cnums = np.zeros(4)
    if enroque_str == "-":
        return cnums
    else:
        if "K" in enroque_str:
            cnums[0] = 1
        if "Q" in enroque_str:
            cnums[1] = 1
        if "k" in enroque_str:
            cnums[2] = 1
        if "q" in enroque_str:
            cnums[3] = 1
    return cnums

In [ ]:
def create_row(game_state):
    try:
        comment = re.match("\[%eval (.*)\]", game_state.comment).groups()[0]
    except:
        return None
    tablero_fen = game_state.board().fen()
    tablero, turno, enroque, _, halfmove, fullmove = analizar_fen(tablero_fen)
    return fullmove, tablero_a_nums(tablero), comment

In [ ]:
import re
re.match("\[%eval (.*)\]", game.end().comment).groups()[0]

'#-2'

In [ ]:
def create_dataset(game):
    data = []
    state = game.end()
    while True:
        row = create_row(state)
        if row is not None:
            fullmove, tablero, comment = row
            if comment[0] != "#":
                data.append(np.append(tablero, [comment]))
            if comment[0] == "#":
                if comment[1] == "-":
                    data.append(np.append(tablero, [-(50 + int(comment[2:])) * 100]))
                else:
                    data.append(np.append(tablero, [(50 - int(comment[1:])) * 100]))
            if int(fullmove) == 1:
                break
        try:
            state = state.parent
        except:
            break
    return np.asarray(data)

In [ ]:
dataset = []
from tqdm import tqdm

for game in tqdm(juegos):
    game_pgn = StringIO("\n".join(game))
    try:
        game = chess.pgn.read_game(game_pgn)
        data = create_dataset(game)
        if len(data.shape) == 2:
            dataset.append(data)
    except:
        continue

Se han truncado las últimas 5000 líneas del flujo de salida.
ValueError: illegal san: 'c6' in 8/1k6/p1R5/P1R5/8/5P2/1r4PP/3r2K1 w - - 9 35
error during pgn parsing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/chess/pgn.py", line 1052, in read_game
    move = board_stack[-1].parse_san(token)
  File "/usr/local/lib/python3.7/dist-packages/chess/__init__.py", line 2612, in parse_san
    raise ValueError("illegal san: {} in {}".format(repr(san), self.fen()))
ValueError: illegal san: 'Qa4' in 8/1k6/p1R5/P1R5/8/5P2/1r4PP/3r2K1 w - - 9 35
error during pgn parsing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/chess/pgn.py", line 1052, in read_game
    move = board_stack[-1].parse_san(token)
  File "/usr/local/lib/python3.7/dist-packages/chess/__init__.py", line 2612, in parse_san
    raise ValueError("illegal san: {} in {}".format(repr(san), self.fen()))
ValueError: illegal san: 'Bd7' in 8/1k6/p1R5/P1R5/8/5P2/1r4PP/3r2K1 

In [ ]:
len(dataset)

2156

In [ ]:
dataset = np.concatenate(dataset)

In [ ]:
dataset.shape

(164431, 65)

In [ ]:
np.savetxt("/content/drive/My Drive/Chess/preprocessed.csv", dataset, delimiter=",", fmt="%s")